KcELECTRA 
- 학습으로 한국어 댓글(Korean Comments). 구어체와 같이 잘 정제되어있지 않은 데이터 셋 학습에 유리 


In [1]:
%pip install transformers
%pip install pandas
%pip install torch
%pip install openpyxl

import pandas as pd 
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)


Note: you may need to restart the kernel to use updated packages.


C:\Users\doohk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
device: cuda:0


In [2]:
df = pd.read_csv('Dataset.csv', sep='\t')
df.head()

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


In [42]:
import os

#ttdf = pd.read_excel('Dataset/12세 미만/꼬마버스 타요4_학습 (All)/타요1.xlsx')
#print(ttdf.head())
#print(ttdf.iloc[:,4].head())

def load_excel_files(directory):
    excel_files = []
    for path, _, files in os.walk(directory):
        for file in files:
            #print(file)
            if file.endswith('.xlsx'):
                file_path = os.path.join(path, file)
                excel_files.append(file_path)
        #print(path)
        #print(excel_files)
    print(excel_files)
    dataframes = [[] * len(excel_files) for _ in range(len(excel_files)) ]
    for i, file_path in enumerate(excel_files):
        tdf = pd.read_excel(file_path)
        dataframes[i]=tdf
    return dataframes

directory = ["Dataset/12세 미만/", "Dataset/15/", "Dataset/19/"]
dataframes = [[],[],[]]
for i, path in enumerate(directory):
    dataframes[i] = load_excel_files(path)

def print_dataset(data):
    for i in data:
        print("next")
        for j in i:
            print(j.head())
            print("------------------------------------------------------------------------------------------")

print_dataset(dataframes)


['Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대1.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대10.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대11.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대12.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대13.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대2.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대3.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대4.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대5.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대6.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대7.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대8.xlsx', 'Dataset/12세 미만/고고 다이노 공룡탐험대3_학습 (All)\\공룡탐험대9.xlsx', 'Dataset/12세 미만/꼬마버스 타요4_학습 (All)\\타요1.xlsx', 'Dataset/12세 미만/꼬마버스 타요4_학습 (All)\\타요10.xlsx', 'Dataset/12세 미만/꼬마버스 타요4_학습 (All)\\타요11.xlsx', 'Dataset/12세 미만/꼬마버스 타요4_학습 (All)\\타요12.xlsx', 'Dataset/12세 미만/꼬마버스 타요4_학습 (All)\\타요13.xlsx', 'Dataset/12세 미만/꼬마버스 타요4_학습 (All)\\타요2.xlsx', 'Dataset/12세 

In [4]:
df.info()
for i in dataframes:
    for j in i:
        print(j.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  10000 non-null  object 
 1   lable    9975 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Time           315 non-null    object 
 1   Subtitle       315 non-null    object 
 2   label          315 non-null    float64
 3   fixed_label    315 non-null    float64
 4   sigmoid_label  315 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 12.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Time      

In [5]:
null_idx = df[df.lable.isnull()].index
df.loc[null_idx, 'content']

1602    응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654           토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992    "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                 에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720          답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
3807    알겠다이기ㅋㅋ 딱 채찍쳐맞는거 좋아하는 한국식 마인드네. 노예마인드. 조금만 성공한...
3908           이래서 스스로 걸리거든 "죄인들이"~ㅎㅎㅎ 재미보고 털리고 그치~~~?\t0
4241    아버지는 내재된 악마들을 다룰 정신적 힘을 가지고 있지 않았다." 이 말한마디가 사...
4283    댓글 중 "선동 당해서 촞불든 개돼지 홍어들도 단죄를 받아야 할 공범자들이다"에10...
5000    스파이 제안받고 살해 안당하는 법1. 처음에 스파이 제안을 받았을때 "중국을 위해서...
5521    "국방부 "까지 ㅡㄱ ㅐ 엿같은 ㅈ ㅣ랄주댕이...좌빨에서 ㅡ인민군대로 ㅡ가려는건가...
5866    쌩뚱맞게 60대최반엌 치매라니 그것도 곱게 사는 사모님이- -" 알콜중독도 아니고 ...
6477    페미메퇘지쿵쾅년인 메갈페미들은 니들이 좋아하는 싫어요 ㄱㄱ제발부탁해~~"일반 여성"...
6538    아니 ㅆㅂ 그런 "카더라"가 넘쳐난다고 그거에 대해서 혹시 댓글게이는 뭔가 아는거 ...
6771    저 때 투니버스에서 코요태 짧게 인터뷰 했었는데 김종민이 "노래는 뭐 신지가 다 하...
6932               개 족 가튼 국방부의 "휴기연장콜센터"발족을 축하한다 ㅆ ㅂ..\t0
7199    민족적 자존심과 애국심을 갖고 국산품 이용합시다 . . . "겸손"한 마음으로 재산...
7252    아나운서는 

In [6]:
df.loc[null_idx, 'lable'] = df.loc[null_idx, 'content'].apply(lambda x:x[-1])

df.loc[null_idx, 'content'] = df.loc[null_idx,'content'].apply(lambda x:x[:-2])

df.head()

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


In [7]:
df = df.astype({'lable':'int'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10000 non-null  object
 1   lable    10000 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 117.3+ KB


In [8]:
#train:test = 4:1
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [9]:
print('train data include duplicate : {}'.format(len(train_data)))
print('test data include duplicate : {}'.format(len(test_data)))

train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)

print('train data drop replication : {}'.format(len(train_data)))
print('test data drop replication : {}'.format(len(test_data)))

train data include duplicate : 8000
test data include duplicate : 2000
train data drop replication : 7992
test data drop replication : 2000


In [37]:
print(train_data.head())
print(dataframes[1][1].head())

                                                content  lable
6252  국방부~~전화로 휴가연장을 한 병사들 몇이나 되는지 공개해라~ 어느 훌륭한 집안의 ...      1
4684          아니라고 해줘요 함정에 빠진거라고 해줘요ㅠㅠ이런 나쁜일을 할사람 아니잖아요      1
1731  이모현 이런 사항을 예상 못 했다고 ㅅㅂ.. mbc 이제는 하다하다 이런 개같은프로...      0
4742                    왜 김수현은 오래걸리는거같지?? 이민호는 빨리나오는거같네      1
4521                                     신성록이 주인공인가요?ㅋㅋ      1
   Time                  Subtitle     label  fixed-label  label.1
0  1:20       (현진) 야, 씨, 우리만 감았어?  0.985472            1      NaN
1  1:22                자기는 안 감았냐고  0.029872            0      NaN
2  1:26  (현진) 우리보다 더 해 먹었으면 해 먹었지  0.985326            1      NaN
3  1:28  우리 쳐내고 자기 혼자 다 해 먹겠다는 건데  0.035319            0      NaN
4  1:31      그걸 신 회장이 모를 리 없는데 왜?  0.013789            0      NaN


Tokenizing 
- 구두점이나 특수 문자를 단순 제외해서는 안됨.(Don't)
- 줄임말과 단어 내에 띄어쓰기가 있는 경우

In [10]:
MODEL_NAME = 'beomi/KcELECTRA-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors='pt', #pytorch의 tensor 형태로 return 
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [11]:
print(tokenized_train_sentences[0].ids)

[7727, 424, 497, 9927, 6413, 712, 3101, 372, 1566, 5898, 16745, 9129, 11079, 96, 1952, 7846, 3628, 299, 8668, 3819, 10134, 1744, 11079, 424, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [12]:
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors='pt',
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [13]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  
  def __len__(self):
    return len(self.labels)

    

In [14]:
train_label = train_data['lable'].values
test_label = test_data['lable'].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)


Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.ou

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [16]:
%pip install --upgrade accelerate

Note: you may need to restart the kernel to use updated packages.


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [18]:
%pip uninstall -y transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.20.0.dev0
Uninstalling accelerate-0.20.0.dev0:
  Successfully uninstalled accelerate-0.20.0.dev0
Note: you may need to restart the kernel to use updated packages.


In [19]:

%pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to c:\users\doohk\appdata\local\temp\pip-req-build-fb0hb_sh
  Resolved https://github.com/huggingface/accelerate to commit c9fbb71e37e7f64f5df54b39270bdabe82f1b893
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=228108 sha256=9e0427e04f91daf799e835812125c9d054cb3e5833323e9362053873eaf97671
  Stored in directory: C:\Users\doohk\AppData\Local\Temp\pip-ephem-wheel-cache-4gw8yr68\wheels\18\af\f7\facfc4ea8d2484e23fc8489825221fe5826625fad79301dd99
Successfully built accelerate


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate 'C:\Users\doohk\AppData\Local\Temp\pip-req-build-fb0hb_sh'


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2,
)

In [21]:
def compute_metric(pred):
  labels = pred.labels_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy' : acc,
      'f1' : f1,
      'precision': precision,
      'recall': recall
  }

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

In [23]:
trainer.train()

C:\Users\doohk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  0%|          | 0/9990 [00:00<?, ?it/s]C:\Users\doohk\AppData\Local\Temp\ipykernel_20292\389196501.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding

{'loss': 0.3819, 'learning_rate': 4.74974974974975e-05, 'epoch': 0.5}


C:\Users\doohk\AppData\Local\Temp\ipykernel_20292\389196501.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
 10%|█         | 1000/9990 [01:02<09:19, 16.07it/s]

{'loss': 0.3434, 'learning_rate': 4.4994994994994996e-05, 'epoch': 1.0}


C:\Users\doohk\AppData\Local\Temp\ipykernel_20292\389196501.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
 15%|█▌        | 1500/9990 [01:34<08:49, 16.04it/s]

...

C:\Users\doohk\AppData\Local\Temp\ipykernel_20292\389196501.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
 95%|█████████▌| 9500/9990 [16:11<01:19,  6.14it/s]

{'loss': 0.006, 'learning_rate': 2.4524524524524525e-06, 'epoch': 9.51}


C:\Users\doohk\AppData\Local\Temp\ipykernel_20292\389196501.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 9990/9990 [17:33<00:00,  9.49it/s]

{'train_runtime': 1053.1121, 'train_samples_per_second': 75.889, 'train_steps_per_second': 9.486, 'train_loss': 0.08564981201866845, 'epoch': 10.0}


TrainOutput(global_step=9990, training_loss=0.08564981201866845, metrics={'train_runtime': 1053.1121, 'train_samples_per_second': 75.889, 'train_steps_per_second': 9.486, 'train_loss': 0.08564981201866845, 'epoch': 10.0})

test set


In [29]:
import pandas as pd
import numpy as np

excel_name = "수리남_1화_test.xlsx"
test_sample = pd.read_excel("수리남_1화_test.xlsx")
test_sample = test_sample.drop(['Transliteration'], axis=1)
test_sample['label'] = np.nan

# Iterate over each row and update the 'label' values
for i, row in test_sample.iterrows():
    model.eval()

    token_ = tokenizer.encode_plus(
        row['Subtitle'], 
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        return_tensors='pt'
    )

    input_ids = token_['input_ids'].to(device)
    attention_mask = token_['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        logits = logits.detach().cpu()
        probabilities = torch.sigmoid(logits)
        result = probabilities.squeeze().tolist()

    test_sample.loc[i, 'label'] = result[0] 


print(test_sample)

        Time                    Subtitle     label
0        13s                 [긴장감 도는 음악]  0.006610
1        41s                  [새가 끼룩거린다]  0.990427
2        55s        (인구) 수리남이라는 나라를 아는가?  0.008766
3        58s                  아마 생소할 것이다  0.006881
4       1:00                    남아메리카 대륙  0.010387
..       ...                         ...       ...
957    57:03  (경찰) [영어] 멈춰 [네덜란드어] 손 들어!  0.007051
958    57:07              (경찰) [영어] 돌아서!  0.009509
959    57:08                 [개가 연신 짖는다]  0.989767
960    57:13               [사이렌이 계속 울린다]  0.006816
961  1:00:45                      자막: 삼우  0.009554

[962 rows x 3 columns]


In [31]:

test_sample.to_excel('Result_'+excel_name, index=False)